In [1]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.Dataset
import org.apache.spark.sql.functions._

// Initialize a Spark Session
val sparkSession = SparkSession.builder.appName("Assignment3").getOrCreate()

Intitializing Scala interpreter ...

Spark Web UI available at http://d389638e15a8:4041
SparkContext available as 'sc' (version = 3.5.1, master = local[*], app id = local-1734949959484)
SparkSession available as 'spark'


import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.Dataset
import org.apache.spark.sql.functions._
sparkSession: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@5d5a2103


In [2]:
// Task 1: Disable automatic broadcast join

sparkSession.conf.set("spark.sql.autoBroadcastJoinThreshold", "-1")

In [3]:
// Task 2: Explicitly Broadcast Join medals and maps
// Define required case classes that represent schemas for the data

case class Matches (
    match_id: Option[String],
    mapid: String,
    is_team_game: Boolean,
    playlist_id: String,
    game_variant_id: String,
    is_match_over: Boolean,
    completion_date: String,
    match_duration: String,
    game_mode: String,
    map_variant_id: String
)

case class MatchDetails (
    match_id: Option[String],
    player_gamertag: String,
    previous_spartan_rank: Integer,
    spartan_rank: Integer,
    previous_total_xp: Integer,
    total_xp: Integer,
    previous_csr_tier: Integer,
    previous_csr_designation: Integer,
    previous_csr: Integer,
    previous_csr_percent_to_next_tier: Integer,
    previous_csr_rank: Integer,
    current_csr_tier: Integer,
    current_csr_designation: Integer,
    current_csr: Integer,
    current_csr_percent_to_next_tier: Integer,
    current_csr_rank: Integer,
    player_rank_on_team: Integer,
    player_finished: Boolean,
    player_average_life: String,
    player_total_kills: Integer,
    player_total_headshots: Integer,
    player_total_weapon_damage: Double,
    player_total_shots_landed: Double,
    player_total_melee_kills: Double,
    player_total_melee_damage: Double,
    player_total_assassinations: Double,
    player_total_ground_pound_kills: Double,
    player_total_shoulder_bash_kills: Double,
    player_total_grenade_damage: Double,
    player_total_power_weapon_damage: Double,
    player_total_power_weapon_grabs: Double,
    player_total_deaths: Double,
    player_total_assists: Double,
    player_total_grenade_kills: Double,
    did_win: Double,
    team_id: Double
)

case class Maps (
    mapid: Option[String],
    name: String,
    description: String
)

case class Medals (
    medal_id: Option[Long], // Updated to Long
    sprite_uri: String,
    sprite_left: Integer,
    sprite_top: Integer,
    sprite_sheet_width: Integer,
    sprite_sheet_height: Integer,
    sprite_width: Integer,
    sprite_height: Integer,
    classification: String,
    description: String,
    name: String,
    difficulty: Integer
)

case class MedalsMatchesPlayers (
    match_id: Option[String],
    player_gamertag: String,
    medal_id: Long,
    count: Integer
)

defined class Matches
defined class MatchDetails
defined class Maps
defined class Medals
defined class MedalsMatchesPlayers


In [4]:
// Read all required .csv files into datasets

val matches: Dataset[Matches] = sparkSession.read.option("header", "true")
                                .option("inferSchema", "true")
                                .csv("/home/iceberg/data/matches.csv")
                                .as[Matches]

//matches.show(2)

matches: org.apache.spark.sql.Dataset[Matches] = [match_id: string, mapid: string ... 8 more fields]


In [5]:
val matchDetails: Dataset[MatchDetails] = sparkSession.read.option("header", "true")
                                .option("inferSchema", "true")
                                .csv("/home/iceberg/data/match_details.csv")
                                .as[MatchDetails]

//matchDetails.show(2)

matchDetails: org.apache.spark.sql.Dataset[MatchDetails] = [match_id: string, player_gamertag: string ... 34 more fields]


In [6]:
val maps: Dataset[Maps] = sparkSession.read.option("header", "true")
                                .option("inferSchema", "true")
                                .csv("/home/iceberg/data/maps.csv")
                                .as[Maps]
//maps.show(2)

maps: org.apache.spark.sql.Dataset[Maps] = [mapid: string, name: string ... 1 more field]


In [7]:
val medals: Dataset[Medals] = sparkSession.read.option("header", "true")
                                .option("inferSchema", "true")
                                .csv("/home/iceberg/data/medals.csv")
                                .as[Medals]
//medals.show(2)

medals: org.apache.spark.sql.Dataset[Medals] = [medal_id: bigint, sprite_uri: string ... 10 more fields]


In [8]:
val medalsMatchesPlayers: Dataset[MedalsMatchesPlayers] = sparkSession.read.option("header", "true")
                                .option("inferSchema", "true")
                                .csv("/home/iceberg/data/medals_matches_players.csv")
                                .as[MedalsMatchesPlayers]
//medalsMatchesPlayers.show(2)                            

medalsMatchesPlayers: org.apache.spark.sql.Dataset[MedalsMatchesPlayers] = [match_id: string, player_gamertag: string ... 2 more fields]


In [9]:
// Create Temporary Views for the required datasets

matches.createOrReplaceTempView("matchesView")

matchDetails.createOrReplaceTempView("matchDetailsView")

maps.createOrReplaceTempView("mapsView")

medalsMatchesPlayers.createOrReplaceTempView("medalsMatchesPlayersView")

medals.createOrReplaceTempView("medalsView")

In [10]:
// Create the long table (medals) using sql

val matchesMedalMapsAgg = sparkSession.sql(f"""
                                            SELECT mmp.medal_id,
                                                    me.name medal_name,
                                                    mmp.match_id,
                                                    ma.mapid,
                                                    COLLECT_LIST(DISTINCT mmp.player_gamertag) as player_gamertag_array
                                                    
                                            FROM medalsMatchesPlayersView mmp
                                            
                                            JOIN matchesView ma
                                            ON mmp.match_id = ma.match_id
                                            
                                            JOIN medalsView me
                                            ON mmp.medal_id = me.medal_id
                                        
                                            GROUP BY 1,2,3,4
                                        """) //.cache()
matchesMedalMapsAgg.show(5)

+---------+----------+--------------------+--------------------+---------------------+
| medal_id|medal_name|            match_id|               mapid|player_gamertag_array|
+---------+----------+--------------------+--------------------+---------------------+
|121048710|   Rampage|073a5745-339b-4f7...|c74c9d0f-f206-11e...|              [Lanqe]|
|121048710|   Rampage|143a653a-27f6-4d2...|c74c9d0f-f206-11e...|     [False EnvisioN]|
|121048710|   Rampage|259119e2-e79c-443...|c7805740-f206-11e...|     [Lvl 61 Scyther]|
|121048710|   Rampage|3453a66d-6a56-434...|cebd854f-f206-11e...|    [BiscuitAnanas31]|
|121048710|   Rampage|35e30ec5-7e71-43c...|c7b7baf0-f206-11e...|        [XCornholeoX]|
+---------+----------+--------------------+--------------------+---------------------+
only showing top 5 rows



matchesMedalMapsAgg: org.apache.spark.sql.DataFrame = [medal_id: bigint, medal_name: string ... 3 more fields]


In [11]:
// Explicitly broadcast join Medals & Maps (Below is a Dataframe. It doesn't conform to a case class)

val medalsMaps = matchesMedalMapsAgg.as("m")
                    .join(broadcast(maps).as("mp"), $"m.mapid" === $"mp.mapid")
                    .select($"m.*", $"mp.name".as("map_name"), $"mp.description".as("map_description"))

medalsMaps.show(5)

+---------+----------+--------------------+--------------------+---------------------+--------+--------------------+
| medal_id|medal_name|            match_id|               mapid|player_gamertag_array|map_name|     map_description|
+---------+----------+--------------------+--------------------+---------------------+--------+--------------------+
|121048710|   Rampage|073a5745-339b-4f7...|c74c9d0f-f206-11e...|              [Lanqe]|  Alpine|These vistas are ...|
|121048710|   Rampage|143a653a-27f6-4d2...|c74c9d0f-f206-11e...|     [False EnvisioN]|  Alpine|These vistas are ...|
|121048710|   Rampage|259119e2-e79c-443...|c7805740-f206-11e...|     [Lvl 61 Scyther]| Glacier|Each of Halo's mi...|
|121048710|   Rampage|3453a66d-6a56-434...|cebd854f-f206-11e...|    [BiscuitAnanas31]|Coliseum|Forerunner Warrio...|
|121048710|   Rampage|35e30ec5-7e71-43c...|c7b7baf0-f206-11e...|        [XCornholeoX]|Parallax|The Orion Arm of ...|
+---------+----------+--------------------+--------------------+

medalsMaps: org.apache.spark.sql.DataFrame = [medal_id: bigint, medal_name: string ... 5 more fields]


In [12]:
// Task 3: Bucket join match_details, matches, and medal_matches_players on match_id with 16 buckets

// Create DDL for bucketed tables

// Matches
sparkSession.sql("""DROP TABLE IF EXISTS bootcamp.matches_bucketed""")

val bucketedMatchesDDL = """
                            CREATE TABLE IF NOT EXISTS bootcamp.matches_bucketed (
                                match_id STRING,
                                is_team_game BOOLEAN,
                                playlist_id STRING,
                                completion_date TIMESTAMP
                            )
                            USING iceberg
                            PARTITIONED BY (completion_date, bucket(16, match_id));
                            """
sparkSession.sql(bucketedMatchesDDL)



// Match Details
sparkSession.sql("""DROP TABLE IF EXISTS bootcamp.match_details_bucketed""")

val bucketedMatchDetailsDDL = """
                                CREATE TABLE IF NOT EXISTS bootcamp.match_details_bucketed (
                                    match_id STRING,
                                    player_gamertag STRING,
                                    player_total_kills INTEGER,
                                    player_total_deaths INTEGER
                                )
                                USING iceberg
                                PARTITIONED BY (bucket(16, match_id));
                                """
sparkSession.sql(bucketedMatchDetailsDDL)



// Medals Matches Players
sparkSession.sql("""DROP TABLE IF EXISTS bootcamp.medal_matches_players_bucketed""")

val bucketedMedalMatchesPlayersDDL = """
                                        CREATE TABLE IF NOT EXISTS bootcamp.medal_matches_players_bucketed (
                                            match_id STRING,
                                            player_gamertag STRING,
                                            medal_id INTEGER
                                        )
                                        USING iceberg
                                        PARTITIONED BY (bucket(16, match_id));
                                        """
sparkSession.sql(bucketedMedalMatchesPlayersDDL)

bucketedMatchesDDL: String =
"
                            CREATE TABLE IF NOT EXISTS bootcamp.matches_bucketed (
                                match_id STRING,
                                is_team_game BOOLEAN,
                                playlist_id STRING,
                                completion_date TIMESTAMP
                            )
                            USING iceberg
                            PARTITIONED BY (completion_date, bucket(16, match_id));
                            "
bucketedMatchDetailsDDL: String =
"
                                CREATE TABLE IF NOT EXISTS bootcamp.match_details_bucketed (
                                    match_id STRING,
                                    player_gamertag STRING,
                                    player...


In [ ]:
// Write data from the above datasets to the corresponding bucketed tables

// Matches
matches.select($"match_id", $"is_team_game", $"playlist_id", $"completion_date")
        .write.mode("append")
        .partitionBy("completion_date")
        .bucketBy(16, "match_id")
        .saveAsTable("bootcamp.matches_bucketed")

In [ ]:
// Match Details
matchDetails.select($"match_id", $"player_gamertag", $"player_total_kills", $"player_total_deaths")
            .write.mode("append")
            .bucketBy(16, "match_id")
            .saveAsTable("bootcamp.match_details_bucketed")

In [ ]:
// Medals Matches Players
medalsMatchesPlayers.select($"match_id", $"player_gamertag", $"medal_id")
                    .write.mode("append")
                    .bucketBy(16, "match_id")
                    .saveAsTable("bootcamp.medal_matches_players_bucketed")

In [ ]:
// Bucket Join match_details, matches, and medal_matches_players

//val bucketedMatchesMedals = 
sparkSession.sql("""
                SELECT mb.match_id, mb.playlist_id, md.completion_date,
                        mdb.player_gamertag, mdb.player_total_kills, mdb.player_total_deaths,
                        mmpb.medal_id
                        
                FROM bootcamp.matches_bucketed mb 
                            
                JOIN bootcamp.match_details_bucketed mdb 
                ON mb.match_id = mdb.match_id
                            
                JOIN bootcamp.medalMatchesPlayersBucketed mmpb
                ON mb.match_id = mmpb.match_id
                AND mb.completion_date = DATE('2016-01-01') 
                """).explain()

//bucketedMatchesMedals.createOrReplaceTempView("bucketed_matches_medals")

In [ ]:
// Join match_details, matches, and medal_matches_players temporary views

sparkSession.sql("""
                SELECT mb.match_id, mb.playlist_id, md.completion_date,
                        mdb.player_gamertag, mdb.player_total_kills, mdb.player_total_deaths,
                        mmpb.medal_id
                
                FROM matchesView mb 
                
                JOIN matchDetailsView mdb 
                ON mb.match_id = mdb.match_id
                
                JOIN medalsMatchesPlayersView mmpb
                ON mb.match_id = mmpb.match_id
                
                AND mb.completion_date = DATE('2016-01-01') 
                """).explain()

In [ ]:
// Task 4: Aggregate the joined data frame to figure out questions like:
// I. Which player averages the most kills per game?

sparkSession.sql("""
                SELECT player_gametag, 
                        AVERAGE(player_total_kills) average_kills 
                
                FROM bucketed_matches_medals
                ORDER BY AVERAGE(player_total_kills) DESC
                """)

In [ ]:
// %%sql
// SELECT player_gametag, AVERAGE(player_total_kills) average_kills FROM bucketed_matches_medals ORDER BY AVERAGE(player_total_kills) DESC

In [ ]:
// II. Which playlist gets played the most?
        // III. Which map gets played the most?
        // IV. Which map do players get the most Killing Spree medals on?